In [15]:
import numpy as np
import pandas as pd


wd="/mnt/storage5/anton/Mosquitoes/ResistanceGenes/"

vcfFileName="AnnotatedResistanceV2.vcf"

#Process bed file used to get subregions for mapping to label each base in the VCF with Gene, Exon/Intron, CDS, Gene Name
positionMetaData={} #key=chromosome, value={key=position, value={"Intron/Exon":, "isCDS":, "GeneID":} }
relevantGeneIDs=set() #these are gene IDs to which reads have been aligned.
with open(wd+"InputData/mappedRegions.bed") as bedFile:
    for line in bedFile:
        values=line.strip().split("\t")
        relevantGeneIDs.add(values[3])
        if values[0] not in positionMetaData:
            positionMetaData[values[0]]={}
        for i in range(int(values[1]), int(values[2])):
            positionMetaData[values[0]][i]={"mRNA":False,"exon":False, "CDS":False, "GeneID":values[3]}
##Populate the isExon, isCDS for the positionMetaData


with open("/mnt/storage5/anton/Mosquitoes/GCF_002204515.2_AaegL5.0_genomic.gff") as gffFile:
    for line in gffFile:
        if line[0]!="#":
            values=line.strip().split("\t")
            if values[2] in ["mRNA","exon","CDS"]:
                for relevantGeneID in relevantGeneIDs:
                    if values[8].find(relevantGeneID)>-1: #check if id (ID=rna-XM_021846286.1) is among the target genes:
                        for i in range( int(values[3]),  int(values[4])+1  ):
                            positionMetaData[values[0]][i][values[2]]=True
                        break


In [16]:
gene_metadata=pd.read_csv(f'{wd}/InputData/GenesList.tsv', sep="\t", index_col=0)

In [25]:
ExcludeRegions=[]
ExcludeSamples=[]
include_indices=[]

cds_only_file=open(f'{wd}/OutputData/AnnotatedResistanceV3_exon_only.vcf',"w")
with open(wd+vcfFileName) as full_vcf_file:
    for line in full_vcf_file:
        if line[0:6]=="#CHROM": #get indices of samples that need to be kept
            for i, value in enumerate(line.strip().split("\t")):
                if i<9:
                    include_indices.append(i)
                else:
                    if value not in ExcludeSamples:
                        include_indices.append(i)
            include_indices=np.asarray(include_indices)
            cds_only_file.write('\t'.join(np.asarray(line.strip().split("\t"))[include_indices])+"\n")
        elif line[0]=="#" and line[0:6]!="#CHROM":
            cds_only_file.write(line)
        else:
            values=line.strip().split("\t")
            if values[0]=="NC_035159.1" or positionMetaData[values[0]][int(values[1])]["exon"]:#cox1 has no start codon in reference, hence why it's accomodated separately     
                position_coordinate=str(values[0])+"_"+str(values[1])
                cds_only_file.write('\t'.join(np.asarray(values)[include_indices])+"\n")

